In [3]:
# -*- coding: utf-8 -*-
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np # 数据处理的库 numpy
import argparse
import imutils
import time
import dlib
import cv2
 
 
def eye_aspect_ratio(eye):
    # 垂直眼标志（X，Y）坐标
    A = dist.euclidean(eye[1], eye[5])# 计算两个集合之间的欧式距离
    B = dist.euclidean(eye[2], eye[4])
    # 计算水平之间的欧几里得距离
    # 水平眼标志（X，Y）坐标
    C = dist.euclidean(eye[0], eye[3])
    # 眼睛长宽比的计算
    ear = (A + B) / (2.0 * C)
    # 返回眼睛的长宽比
    return ear
 
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[2] - mouth[9])  # 51, 59
    B = np.linalg.norm(mouth[4] - mouth[7])  # 53, 57
    C = np.linalg.norm(mouth[0] - mouth[6])  # 49, 55
    mar = (A + B) / (2.0 * C)
    return mar


# 定义两个常数
# 眼睛长宽比
# 闪烁阈值
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3
# 打哈欠长宽比
# 闪烁阈值
MAR_THRESH = 0.5
MOUTH_AR_CONSEC_FRAMES = 3
# 初始化帧计数器和眨眼总数
COUNTER = 0
TOTAL = 0
# 初始化帧计数器和打哈欠总数
mCOUNTER = 0
mTOTAL = 0
 
# 初始化DLIB的人脸检测器（HOG），然后创建面部标志物预测
print("[INFO] loading facial landmark predictor...")
# 第一步：使用dlib.get_frontal_face_detector() 获得脸部位置检测器
detector = dlib.get_frontal_face_detector()
# 第二步：使用dlib.shape_predictor获得脸部特征位置检测器
predictor = dlib.shape_predictor('D:/myworkspace/JupyterNotebook/fatigue_detecting/model/shape_predictor_68_face_landmarks.dat')
 
# 第三步：分别获取左右眼面部标志的索引
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# 第四步：打开cv2 本地摄像头
cap = cv2.VideoCapture(0)
 
# 从视频流循环帧
while True:
    # 第五步：进行循环，读取图片，并对图片做维度扩大，并进灰度化
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=720)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 第六步：使用detector(gray, 0) 进行脸部位置检测
    rects = detector(gray, 0)
    
    # 第七步：循环脸部位置信息，使用predictor(gray, rect)获得脸部特征位置的信息
    for rect in rects:
        shape = predictor(gray, rect)
        
        # 第八步：将脸部特征信息转换为数组array的格式
        shape = face_utils.shape_to_np(shape)
        
        # 第九步：提取左眼和右眼坐标
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        # 嘴巴坐标
        mouth = shape[mStart:mEnd]
        
        
        # 第十步：构造函数计算左右眼的EAR值，使用平均值作为最终的EAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        # 打哈欠
        mar = mouth_aspect_ratio(mouth)
 
        # 第十一步：使用cv2.convexHull获得凸包位置，使用drawContours画出轮廓位置进行画图操作
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
 
        # 第十二步：进行画图操作，用矩形框标注人脸
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)    
 
        '''
            分别计算左眼和右眼的评分求平均作为最终的评分，如果小于阈值，则加1，如果连续3次都小于阈值，则表示进行了一次眨眼活动
        '''
        # 第十三步：循环，满足条件的，眨眼次数+1
        if ear < EYE_AR_THRESH:# 眼睛长宽比：0.2
            COUNTER += 1
           
        else:
            # 如果连续3次都小于阈值，则表示进行了一次眨眼活动
            if COUNTER >= EYE_AR_CONSEC_FRAMES:# 阈值：3
                TOTAL += 1
            # 重置眼帧计数器
            COUNTER = 0
            
        # 第十四步：进行画图操作，同时使用cv2.putText将眨眼次数进行显示
        cv2.putText(frame, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "COUNTER: {}".format(COUNTER), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (450, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        '''
            计算张嘴评分，如果小于阈值，则加1，如果连续3次都小于阈值，则表示打了一次哈欠，同一次哈欠大约在3帧
        '''
        # 同理，判断是否打哈欠    
        if mar > MAR_THRESH:# 张嘴阈值0.5
            mCOUNTER += 1
            cv2.putText(frame, "Yawning!", (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # 如果连续3次都小于阈值，则表示打了一次哈欠
            if mCOUNTER >= MOUTH_AR_CONSEC_FRAMES:# 阈值：3
                mTOTAL += 1
            # 重置嘴帧计数器
            mCOUNTER = 0
        cv2.putText(frame, "Yawning: {}".format(mTOTAL), (150, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "mCOUNTER: {}".format(mCOUNTER), (300, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
        cv2.putText(frame, "MAR: {:.2f}".format(mar), (480, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
        # 第十五步：进行画图操作，68个特征点标识
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

    print('嘴巴实时长宽比:{:.2f} '.format(mar)+"\t是否张嘴："+str([False,True][mar > MAR_THRESH]))
    print('眼睛实时长宽比:{:.2f} '.format(ear)+"\t是否眨眼："+str([False,True][COUNTER>=1]))
    
    # 确定疲劳提示
    if TOTAL >= 50 or mTOTAL>=15:
        cv2.putText(frame, "SLEEP!!!", (100, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        
    # 按q退出
    cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
    # 窗口显示 show with opencv
    cv2.imshow("Frame", frame)
    
    # if the `q` key was pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# 释放摄像头 release camera
cap.release()
# do a bit of cleanup
cv2.destroyAllWindows()


[INFO] loading facial landmark predictor...


D:\mydownload\Anaconda3\lib\site-packages\ipykernel_launcher.py:152: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


嘴巴实时长宽比:0.52 	是否张嘴：True
眼睛实时长宽比:0.28 	是否眨眼：False
嘴巴实时长宽比:0.53 	是否张嘴：True
眼睛实时长宽比:0.30 	是否眨眼：False
嘴巴实时长宽比:0.53 	是否张嘴：True
眼睛实时长宽比:0.30 	是否眨眼：False
嘴巴实时长宽比:0.51 	是否张嘴：True
眼睛实时长宽比:0.30 	是否眨眼：False
嘴巴实时长宽比:0.51 	是否张嘴：True
眼睛实时长宽比:0.31 	是否眨眼：False
嘴巴实时长宽比:0.51 	是否张嘴：True
眼睛实时长宽比:0.31 	是否眨眼：False
嘴巴实时长宽比:0.53 	是否张嘴：True
眼睛实时长宽比:0.31 	是否眨眼：False
嘴巴实时长宽比:0.51 	是否张嘴：True
眼睛实时长宽比:0.29 	是否眨眼：False
嘴巴实时长宽比:0.50 	是否张嘴：True
眼睛实时长宽比:0.32 	是否眨眼：False
嘴巴实时长宽比:0.49 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
嘴巴实时长宽比:0.50 	是否张嘴：True
眼睛实时长宽比:0.23 	是否眨眼：False
嘴巴实时长宽比:0.51 	是否张嘴：True
眼睛实时长宽比:0.30 	是否眨眼：False
嘴巴实时长宽比:0.49 	是否张嘴：False
眼睛实时长宽比:0.29 	是否眨眼：False
嘴巴实时长宽比:0.54 	是否张嘴：True
眼睛实时长宽比:0.29 	是否眨眼：False
嘴巴实时长宽比:0.53 	是否张嘴：True
眼睛实时长宽比:0.31 	是否眨眼：False
嘴巴实时长宽比:0.51 	是否张嘴：True
眼睛实时长宽比:0.29 	是否眨眼：False
嘴巴实时长宽比:0.52 	是否张嘴：True
眼睛实时长宽比:0.29 	是否眨眼：False
嘴巴实时长宽比:0.50 	是否张嘴：True
眼睛实时长宽比:0.29 	是否眨眼：False
嘴巴实时长宽比:0.53 	是否张嘴：True
眼睛实时长宽比:0.30 	是否眨眼：False
嘴巴实时长宽比:0.54 	是否张嘴：True
眼睛实时长宽比:0.30 	是否眨眼：False
嘴巴实时长宽比:0.54 	是否张嘴

In [7]:
#!/usr/bin/env python
 
import cv2
import numpy as np
import dlib
import time
import math

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("D:/myworkspace/JupyterNotebook/fatigue_detecting/model/shape_predictor_68_face_landmarks.dat")
POINTS_NUM_LANDMARK = 68

# 获取最大的人脸
def _largest_face(dets):
    if len(dets) == 1:
        return 0

    face_areas = [ (det.right()-det.left())*(det.bottom()-det.top()) for det in dets]

    largest_area = face_areas[0]
    largest_index = 0
    for index in range(1, len(dets)):
        if face_areas[index] > largest_area :
            largest_index = index
            largest_area = face_areas[index]

    print("largest_face index is {} in {} faces".format(largest_index, len(dets)))

    return largest_index

# 从dlib的检测结果抽取姿态估计需要的点坐标
def get_image_points_from_landmark_shape(landmark_shape):
    if landmark_shape.num_parts != POINTS_NUM_LANDMARK:
        print("ERROR:landmark_shape.num_parts-{}".format(landmark_shape.num_parts))
        return -1, None
    
    #2D image points. If you change the image, you need to change vector
    image_points = np.array([
                                (landmark_shape.part(30).x, landmark_shape.part(30).y),     # Nose tip
                                (landmark_shape.part(8).x, landmark_shape.part(8).y),     # Chin
                                (landmark_shape.part(36).x, landmark_shape.part(36).y),     # Left eye left corner
                                (landmark_shape.part(45).x, landmark_shape.part(45).y),     # Right eye right corne
                                (landmark_shape.part(48).x, landmark_shape.part(48).y),     # Left Mouth corner
                                (landmark_shape.part(54).x, landmark_shape.part(54).y)      # Right mouth corner
                            ], dtype="double")

    return 0, image_points
    
# 用dlib检测关键点，返回姿态估计需要的几个点坐标
def get_image_points(img):
                            
    #gray = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY )  # 图片调整为灰色
    dets = detector( img, 0 )

    if 0 == len( dets ):
        print( "ERROR: found no face" )
        return -1, None
    largest_index = _largest_face(dets)
    face_rectangle = dets[largest_index]

    landmark_shape = predictor(img, face_rectangle)

    return get_image_points_from_landmark_shape(landmark_shape)


# 获取旋转向量和平移向量                        
def get_pose_estimation(img_size, image_points ):
    # 参考：https://www.learnopencv.com/head-pose-estimation-using-opencv-and-dlib/
    # 3D model points.
    model_points = np.array([
                                (0.0, 0.0, 0.0),             # Nose tip
                                (0.0, -330.0, -65.0),        # Chin
                                (-225.0, 170.0, -135.0),     # Left eye left corner
                                (225.0, 170.0, -135.0),      # Right eye right corne
                                (-150.0, -150.0, -125.0),    # Left Mouth corner
                                (150.0, -150.0, -125.0)      # Right mouth corner
                             
                            ])
     
    # Camera internals
     
    focal_length = img_size[1]
    center = (img_size[1]/2, img_size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
     
    print("Camera Matrix :{}".format(camera_matrix))
     
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE )
 
    print("Rotation Vector:\n {}".format(rotation_vector))
    print("Translation Vector:\n {}".format(translation_vector))
    return success, rotation_vector, translation_vector, camera_matrix, dist_coeffs

# 从旋转向量转换为欧拉角
def get_euler_angle(rotation_vector):
    # calculate rotation angles
    theta = cv2.norm(rotation_vector, cv2.NORM_L2)
    
    # transformed to quaterniond
    w = math.cos(theta / 2)
    x = math.sin(theta / 2)*rotation_vector[0][0] / theta
    y = math.sin(theta / 2)*rotation_vector[1][0] / theta
    z = math.sin(theta / 2)*rotation_vector[2][0] / theta
    
    ysqr = y * y
    # pitch (x-axis rotation)
    t0 = 2.0 * (w * x + y * z)
    t1 = 1.0 - 2.0 * (x * x + ysqr)
    print('t0:{}, t1:{}'.format(t0, t1))
    pitch = math.atan2(t0, t1)
    
    # yaw (y-axis rotation)
    t2 = 2.0 * (w * y - z * x)
    if t2 > 1.0:
        t2 = 1.0
    if t2 < -1.0:
        t2 = -1.0
    yaw = math.asin(t2)
    
    # roll (z-axis rotation)
    t3 = 2.0 * (w * z + x * y)
    t4 = 1.0 - 2.0 * (ysqr + z * z)
    roll = math.atan2(t3, t4)
    
    print('pitch:{}, yaw:{}, roll:{}'.format(pitch, yaw, roll))
    
	# 单位转换：将弧度转换为度
    Y = int((pitch/math.pi)*180)
    X = int((yaw/math.pi)*180)
    Z = int((roll/math.pi)*180)
    
    return 0, Y, X, Z

def get_pose_estimation_in_euler_angle(landmark_shape, im_szie):
    try:
        ret, image_points = get_image_points_from_landmark_shape(landmark_shape)
        if ret != 0:
            print('get_image_points failed')
            return -1, None, None, None
    
        ret, rotation_vector, translation_vector, camera_matrix, dist_coeffs = get_pose_estimation(im_szie, image_points)
        if ret != True:
            print('get_pose_estimation failed')
            return -1, None, None, None
    
        ret, pitch, yaw, roll = get_euler_angle(rotation_vector)
        if ret != 0:
            print('get_euler_angle failed')
            return -1, None, None, None

        euler_angle_str = 'Y:{}, X:{}, Z:{}'.format(pitch, yaw, roll)
        print(euler_angle_str)
        return 0, pitch, yaw, roll
    
    except Exception as e:
        print('get_pose_estimation_in_euler_angle exception:{}'.format(e))
        return -1, None, None, None
        
if __name__ == '__main__':

    cap = cv2.VideoCapture(0)
    while (cap.isOpened()):
        start_time = time.time()
        
        # Read Image
        ret, im = cap.read()
        if ret != True:
            print('read frame failed')
            continue
        size = im.shape
        
        if size[0] > 700:
            h = size[0] / 3
            w = size[1] / 3
            im = cv2.resize( im, (int( w ), int( h )), interpolation=cv2.INTER_CUBIC )
            size = im.shape
     
        ret, image_points = get_image_points(im)
        if ret != 0:
            print('get_image_points failed')
            continue
        
        ret, rotation_vector, translation_vector, camera_matrix, dist_coeffs = get_pose_estimation(size, image_points)
        if ret != True:
            print('get_pose_estimation failed')
            continue
        used_time = time.time() - start_time
        print("used_time:{} sec".format(round(used_time, 3)))
        
        ret, pitch, yaw, roll = get_euler_angle(rotation_vector)
        euler_angle_str = 'Y:{}, X:{}, Z:{}'.format(pitch, yaw, roll)
        print(euler_angle_str)
        
        # Project a 3D point (0, 0, 1000.0) onto the image plane.
        # We use this to draw a line sticking out of the nose
         
        (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)
         
        for p in image_points:
            cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
         
         
        p1 = ( int(image_points[0][0]), int(image_points[0][1]))
        p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
         
        cv2.line(im, p1, p2, (255,0,0), 2)
         
        # Display image
        #cv2.putText( im, str(rotation_vector), (0, 100), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1 )
        cv2.putText( im, euler_angle_str, (0, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1 )
        # 按q退出
        cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
        # 窗口显示 show with opencv
        cv2.imshow("Output", im)
        # if the `q` key was pressed, break from the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # 释放摄像头 release camera
    cap.release()
    # do a bit of cleanup
    cv2.destroyAllWindows()



Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.28795339]
 [ 0.12774694]
 [-0.25047449]]
Translation Vector:
 [[ 341.79770399]
 [  47.68323618]
 [3389.54417784]]
used_time:0.554 sec
t0:0.15128660684155779, t1:-0.9760368099084753
pitch:2.987815451701243, yaw:-0.15705622270622585, roll:-0.06554251739732628
Y:171, X:-8, Z:-3
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.28971491]
 [ 0.12447173]
 [-0.25155751]]
Translation Vector:
 [[ 340.18522843]
 [  47.22105966]
 [3380.54348181]]
used_time:0.068 sec
t0:0.15310565776403243, t1:-0.9756796429682955
pitch:2.9859399373956057, yaw:-0.1575196275834502, roll:-0.06332788694516155
Y:171, X:-9, Z:-3
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.28295523]
 [ 0.13008136]
 [-0.34468586]]
Translation Vector:
 [[ 346.40066467]
 [  45.07729256]
 [3382.07540854]]
used_time:0.078 sec
t0:0.15209351105738667, t1:-0.96528

Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.28306819]
 [ 0.16279422]
 [ 0.10882999]]
Translation Vector:
 [[ 271.65789384]
 [  36.40059662]
 [3428.44023691]]
used_time:0.105 sec
t0:0.14977575107568677, t1:-0.9869913991042139
pitch:2.9909918473486323, yaw:0.05847304892398459, roll:-0.10350347731981607
Y:171, X:3, Z:-5
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.26816481]
 [ 0.18997709]
 [ 0.39416603]]
Translation Vector:
 [[ 209.0705659]
 [  34.325029 ]
 [3406.2166687]]
used_time:0.062 sec
t0:0.16742446820869486, t1:-0.9594908189974014
pitch:2.9688389268080946, yaw:0.22858286162478464, roll:-0.13600800804623386
Y:170, X:13, Z:-7
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.26816481]
 [ 0.18997709]
 [ 0.39416603]]
Translation Vector:
 [[ 209.0705659]
 [  34.325029 ]
 [3406.2166687]]
used_time:0.045 sec
t0:0.16742446820869486, t1:-0.959490818997

Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.32132061]
 [ 0.21256445]
 [ 0.11531085]]
Translation Vector:
 [[ 252.04319243]
 [  25.44192114]
 [3465.85223657]]
used_time:0.047 sec
t0:0.19129197122891844, t1:-0.9799058808234742
pitch:2.9488026096636735, yaw:0.05652652673851903, roll:-0.13329288427613023
Y:168, X:3, Z:-7
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.30376774]
 [ 0.17465813]
 [-0.38206629]]
Translation Vector:
 [[ 341.1760781 ]
 [  16.21635771]
 [3423.61373131]]
used_time:0.052 sec
t0:0.17423264995136872, t1:-0.9561460819351266
pitch:2.9613464649492687, yaw:-0.23766178658762213, roll:-0.0840565364751299
Y:169, X:-13, Z:-4
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.28935354]
 [ 0.15528178]
 [-0.57425812]]
Translation Vector:
 [[ 408.66957251]
 [   9.05092293]
 [3374.56634446]]
used_time:0.055 sec
t0:0.1807784179988024, t1:-0.921411

ERROR: found no face
get_image_points failed
ERROR: found no face
get_image_points failed
ERROR: found no face
get_image_points failed
ERROR: found no face
get_image_points failed
ERROR: found no face
get_image_points failed
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.26703002]
 [-0.07270615]
 [-1.05693112]]
Translation Vector:
 [[ 594.44381976]
 [ -58.35824432]
 [3322.29449836]]
used_time:0.045 sec
t0:0.28741975877998155, t1:-0.77203301475392
pitch:2.7852004569727464, yaw:-0.6027156840884625, roll:0.1563480120814033
Y:159, X:-34, Z:8
ERROR: found no face
get_image_points failed
ERROR: found no face
get_image_points failed
ERROR: found no face
get_image_points failed
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.22070645]
 [-0.06447096]
 [-1.08799208]]
Translation Vector:
 [[ 563.38758424]
 [ -49.12029911]
 [3120.75206846]]
used_time:0.046 sec
t0:0.2541021607276515, t1:-0.7653873901954285
pi

Rotation Vector:
 [[-3.34943041]
 [ 0.03533007]
 [-0.22406603]]
Translation Vector:
 [[ 379.24303362]
 [  24.64087102]
 [3464.65985176]]
used_time:0.047 sec
t0:0.21196832181890046, t1:-0.9680610312478077
pitch:2.926032898144324, yaw:-0.1342960742982509, roll:-0.00654303834352674
Y:167, X:-7, Z:0
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.31988795]
 [ 0.07059313]
 [-0.11762269]]
Translation Vector:
 [[ 363.10802463]
 [  31.30605902]
 [3519.89242692]]
used_time:0.05 sec
t0:0.1784940744300436, t1:-0.9811553173247658
pitch:2.9616383273700593, yaw:-0.07405488887098441, roll:-0.03583668845956361
Y:169, X:-4, Z:-2
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.31988795]
 [ 0.07059313]
 [-0.11762269]]
Translation Vector:
 [[ 363.10802463]
 [  31.30605902]
 [3519.89242692]]
used_time:0.047 sec
t0:0.1784940744300436, t1:-0.9811553173247658
pitch:2.9616383273700593, yaw:-0.07405488887098441, roll:-0.03

 [  0.   0.   1.]]
Rotation Vector:
 [[-3.28327743]
 [ 0.03483108]
 [-0.22617822]]
Translation Vector:
 [[ 293.71035447]
 [  56.4284194 ]
 [3478.62326681]]
used_time:0.048 sec
t0:0.14728514037011098, t1:-0.9794308412343129
pitch:2.9923327554733743, yaw:-0.13836245757207496, roll:-0.010854828540539653
Y:171, X:-7, Z:0
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.25535409]
 [ 0.02275428]
 [-0.33575154]]
Translation Vector:
 [[ 298.23524089]
 [  58.86831286]
 [3468.57702414]]
used_time:0.048 sec
t0:0.12862040817256817, t1:-0.9704577793054554
pitch:3.009824795479439, yaw:-0.20557349137470313, roll:-0.0003679939359825929
Y:172, X:-11, Z:0
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.24046147]
 [ 0.04639823]
 [-0.48587037]]
Translation Vector:
 [[ 299.70615552]
 [  56.43560904]
 [3470.61105475]]
used_time:0.047 sec
t0:0.1293215484644595, t1:-0.9470801920131884
pitch:3.005884316766506, yaw:-0.29819

Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.27146731]
 [ 0.12545817]
 [-0.26354755]]
Translation Vector:
 [[ 141.25964582]
 [  73.54626912]
 [3602.72896755]]
used_time:0.063 sec
t0:0.13572241056948348, t1:-0.9769998831228415
pitch:3.003558530969732, yaw:-0.16522151685234931, roll:-0.06521367160456007
Y:172, X:-9, Z:-3
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.27146731]
 [ 0.12545817]
 [-0.26354755]]
Translation Vector:
 [[ 141.25964582]
 [  73.54626912]
 [3602.72896755]]
used_time:0.047 sec
t0:0.13572241056948348, t1:-0.9769998831228415
pitch:3.003558530969732, yaw:-0.16522151685234931, roll:-0.06521367160456007
Y:172, X:-9, Z:-3
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.3007877 ]
 [ 0.13162915]
 [-0.19737582]]
Translation Vector:
 [[ 135.67527872]
 [  70.11338438]
 [3598.1623705 ]]
used_time:0.049 sec
t0:0.16178389757903983, t1:-0.97890

 [  0.   0.   1.]]
Rotation Vector:
 [[-3.20158135]
 [ 0.3569358 ]
 [-0.96996126]]
Translation Vector:
 [[-1.24258668e+00]
 [ 1.00181531e+02]
 [ 3.69829227e+03]]
used_time:0.045 sec
t0:0.1497450314439694, t1:-0.8111131867595915
pitch:2.9590315384700854, yaw:-0.6009124079527919, roll:-0.16535123753678038
Y:169, X:-34, Z:-9
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.23063788]
 [ 0.36077506]
 [-0.94759354]]
Translation Vector:
 [[-2.23098633e-02]
 [ 9.57215265e+01]
 [ 3.66197537e+03]]
used_time:0.059 sec
t0:0.17214323684323862, t1:-0.8159670764444693
pitch:2.933673180113008, yaw:-0.584609252184628, roll:-0.1596506704618879
Y:168, X:-33, Z:-9
Camera Matrix :[[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.1791603 ]
 [ 0.33838397]
 [-0.94606904]]
Translation Vector:
 [[3.38251961e-01]
 [1.01455922e+02]
 [3.70545965e+03]]
used_time:0.045 sec
t0:0.1254174290512102, t1:-0.821975983130582
pitch:2.9901800420328306, ya

In [4]:
# 参考https://github.com/lincolnhard/head-pose-estimation
import cv2
import dlib
import numpy as np
from imutils import face_utils
"""
思路：
    第一步：2D人脸关键点检测；第二步：3D人脸模型匹配；
    第三步：求解3D点和对应2D点的转换关系；第四步：根据旋转矩阵求解欧拉角。
"""

# 加载人脸检测和姿势估计模型（dlib）
face_landmark_path = 'D:/myworkspace/JupyterNotebook/fatigue_detecting/model/shape_predictor_68_face_landmarks.dat'

"""
只要知道世界坐标系内点的位置、像素坐标位置和相机参数就可以搞定旋转和平移矩阵(OpenCV自带函数solvePnp())
"""

# 世界坐标系(UVW)：填写3D参考点，该模型参考http://aifi.isr.uc.pt/Downloads/OpenGL/glAnthropometric3DModel.cpp
object_pts = np.float32([[6.825897, 6.760612, 4.402142],  #33左眉左上角
                         [1.330353, 7.122144, 6.903745],  #29左眉右角
                         [-1.330353, 7.122144, 6.903745], #34右眉左角
                         [-6.825897, 6.760612, 4.402142], #38右眉右上角
                         [5.311432, 5.485328, 3.987654],  #13左眼左上角
                         [1.789930, 5.393625, 4.413414],  #17左眼右上角
                         [-1.789930, 5.393625, 4.413414], #25右眼左上角
                         [-5.311432, 5.485328, 3.987654], #21右眼右上角
                         [2.005628, 1.409845, 6.165652],  #55鼻子左上角
                         [-2.005628, 1.409845, 6.165652], #49鼻子右上角
                         [2.774015, -2.080775, 5.048531], #43嘴左上角
                         [-2.774015, -2.080775, 5.048531],#39嘴右上角
                         [0.000000, -3.116408, 6.097667], #45嘴中央下角
                         [0.000000, -7.415691, 4.070434]])#6下巴角

# 相机坐标系(XYZ)：添加相机内参
K = [6.5308391993466671e+002, 0.0, 3.1950000000000000e+002,
     0.0, 6.5308391993466671e+002, 2.3950000000000000e+002,
     0.0, 0.0, 1.0]# 等价于矩阵[fx, 0, cx; 0, fy, cy; 0, 0, 1]
# 图像中心坐标系(uv)：相机畸变参数[k1, k2, p1, p2, k3]
D = [7.0834633684407095e-002, 6.9140193737175351e-002, 0.0, 0.0, -1.3073460323689292e+000]

# 像素坐标系(xy)：填写凸轮的本征和畸变系数
cam_matrix = np.array(K).reshape(3, 3).astype(np.float32)
dist_coeffs = np.array(D).reshape(5, 1).astype(np.float32)



# 重新投影3D点的世界坐标轴以验证结果姿势
reprojectsrc = np.float32([[10.0, 10.0, 10.0],
                           [10.0, 10.0, -10.0],
                           [10.0, -10.0, -10.0],
                           [10.0, -10.0, 10.0],
                           [-10.0, 10.0, 10.0],
                           [-10.0, 10.0, -10.0],
                           [-10.0, -10.0, -10.0],
                           [-10.0, -10.0, 10.0]])
# 绘制正方体12轴
line_pairs = [[0, 1], [1, 2], [2, 3], [3, 0],
              [4, 5], [5, 6], [6, 7], [7, 4],
              [0, 4], [1, 5], [2, 6], [3, 7]]

def get_head_pose(shape):
    # 填写2D参考点，注释遵循https://ibug.doc.ic.ac.uk/resources/300-W/
    """
      17左眉左上角/21左眉右角/22右眉左上角/26右眉右上角/36左眼左上角/39左眼右上角/42右眼左上角/
      45右眼右上角/31鼻子左上角/35鼻子右上角/48左上角/54嘴右上角/57嘴中央下角/8下巴角
    """
    # 像素坐标集合
    image_pts = np.float32([shape[17], shape[21], shape[22], shape[26], shape[36],
                            shape[39], shape[42], shape[45], shape[31], shape[35],
                            shape[48], shape[54], shape[57], shape[8]])
    """
    用solvepnp或sovlepnpRansac，输入3d点、2d点、相机内参、相机畸变，输出r、t之后
    用projectPoints，输入3d点、相机内参、相机畸变、r、t，输出重投影2d点
    计算原2d点和重投影2d点的距离作为重投影误差
    """
    # solvePnP计算姿势——求解旋转和平移矩阵：
    # rotation_vec表示旋转矩阵，translation_vec表示平移矩阵，cam_matrix与K矩阵对应，dist_coeffs与D矩阵对应。
    _, rotation_vec, translation_vec = cv2.solvePnP(object_pts, image_pts, cam_matrix, dist_coeffs)
    # projectPoints重新投影误差
    reprojectdst, _ = cv2.projectPoints(reprojectsrc, rotation_vec, translation_vec, cam_matrix,dist_coeffs)

    reprojectdst = tuple(map(tuple, reprojectdst.reshape(8, 2)))# 以8行2列显示

    # 计算欧拉角calc euler angle
    # 参考https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#decomposeprojectionmatrix
    rotation_mat, _ = cv2.Rodrigues(rotation_vec)#罗德里格斯公式（将旋转矩阵转换为旋转向量）
    pose_mat = cv2.hconcat((rotation_mat, translation_vec))# 水平拼接，vconcat垂直拼接
    # eulerAngles –可选的三元素矢量，包含三个以度为单位的欧拉旋转角度
    _, _, _, _, _, _, euler_angle = cv2.decomposeProjectionMatrix(pose_mat)# 将投影矩阵分解为旋转矩阵和相机矩阵

    return reprojectdst, euler_angle


def main():
    # return
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Unable to connect to camera.")
        return
    # 检测人脸
    detector = dlib.get_frontal_face_detector()
    # 检测第一个人脸的关键点
    predictor = dlib.shape_predictor(face_landmark_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            face_rects = detector(frame, 0)

            if len(face_rects) > 0:
                # 循环脸部位置信息，使用predictor(gray, rect)获得脸部特征位置的信息
                shape = predictor(frame, face_rects[0])
                # 将脸部特征信息转换为数组array的格式
                shape = face_utils.shape_to_np(shape)
                # 获取头部姿态
                reprojectdst, euler_angle = get_head_pose(shape)
                pitch = format(euler_angle[0, 0])
                yaw = format(euler_angle[1, 0])
                roll = format(euler_angle[2, 0])
                print('pitch:{}, yaw:{}, roll:{}'.format(pitch, yaw, roll))
                
                # 标出68个特征点
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
                    
                # 绘制正方体12轴
                for start, end in line_pairs:
                    cv2.line(frame, reprojectdst[start], reprojectdst[end], (0, 0, 255))
                # 显示角度结果
                cv2.putText(frame, "X: " + "{:7.2f}".format(euler_angle[0, 0]), (20, 20), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), thickness=2)
                cv2.putText(frame, "Y: " + "{:7.2f}".format(euler_angle[1, 0]), (20, 50), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), thickness=2)
                cv2.putText(frame, "Z: " + "{:7.2f}".format(euler_angle[2, 0]), (20, 80), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), thickness=2)    
    
            # 按q退出提示
            cv2.putText(frame, "Press 'q': Quit", (20, 450),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
            # 窗口显示 show with opencv
            cv2.imshow("Head_Posture", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    # 释放摄像头 release camera
    cap.release()
    # do a bit of cleanup
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

pitch:-1.52592486482225, yaw:-3.641155633231405, roll:-1.7343345663356509
pitch:-0.95245073834262, yaw:-0.9839147796335558, roll:-1.2506715126638224
pitch:-0.9174671700786005, yaw:-1.5239128012326761, roll:-1.324680110854094
pitch:-0.45865802124326294, yaw:-2.078850407172968, roll:-1.806950616178291
pitch:-0.5285176270195822, yaw:-1.3066623687256034, roll:-1.4783328399909006
pitch:-1.1614956033115056, yaw:-2.6926970018811933, roll:-1.4442059039867419
pitch:-2.2354935743612674, yaw:-2.715603409927391, roll:-1.3090854510909937
pitch:-2.274517207168189, yaw:-2.215332515744181, roll:-1.357885150152115
pitch:-0.6695672299130452, yaw:-3.064056104936772, roll:-1.3580953659016977
pitch:-1.4596007667679474, yaw:-4.228628583862223, roll:-1.4342447814158774
pitch:-0.6094586134123463, yaw:-2.5001888584668626, roll:-1.426960264223968
pitch:-0.6094586134123463, yaw:-2.5001888584668626, roll:-1.426960264223968
pitch:-1.709734387659002, yaw:-2.822636852142856, roll:-1.3122006468925178
pitch:-2.9688797

In [14]:
# -*- coding: utf-8 -*-
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np # 数据处理的库 numpy
import argparse
import imutils
import time
import dlib
import cv2
import math
import time
from threading import Thread,Event
 
# 世界坐标系(UVW)：填写3D参考点，该模型参考http://aifi.isr.uc.pt/Downloads/OpenGL/glAnthropometric3DModel.cpp
object_pts = np.float32([[6.825897, 6.760612, 4.402142],  #33左眉左上角
                         [1.330353, 7.122144, 6.903745],  #29左眉右角
                         [-1.330353, 7.122144, 6.903745], #34右眉左角
                         [-6.825897, 6.760612, 4.402142], #38右眉右上角
                         [5.311432, 5.485328, 3.987654],  #13左眼左上角
                         [1.789930, 5.393625, 4.413414],  #17左眼右上角
                         [-1.789930, 5.393625, 4.413414], #25右眼左上角
                         [-5.311432, 5.485328, 3.987654], #21右眼右上角
                         [2.005628, 1.409845, 6.165652],  #55鼻子左上角
                         [-2.005628, 1.409845, 6.165652], #49鼻子右上角
                         [2.774015, -2.080775, 5.048531], #43嘴左上角
                         [-2.774015, -2.080775, 5.048531],#39嘴右上角
                         [0.000000, -3.116408, 6.097667], #45嘴中央下角
                         [0.000000, -7.415691, 4.070434]])#6下巴角

# 相机坐标系(XYZ)：添加相机内参
K = [6.5308391993466671e+002, 0.0, 3.1950000000000000e+002,
     0.0, 6.5308391993466671e+002, 2.3950000000000000e+002,
     0.0, 0.0, 1.0]# 等价于矩阵[fx, 0, cx; 0, fy, cy; 0, 0, 1]
# 图像中心坐标系(uv)：相机畸变参数[k1, k2, p1, p2, k3]
D = [7.0834633684407095e-002, 6.9140193737175351e-002, 0.0, 0.0, -1.3073460323689292e+000]

# 像素坐标系(xy)：填写凸轮的本征和畸变系数
cam_matrix = np.array(K).reshape(3, 3).astype(np.float32)
dist_coeffs = np.array(D).reshape(5, 1).astype(np.float32)



# 重新投影3D点的世界坐标轴以验证结果姿势
reprojectsrc = np.float32([[10.0, 10.0, 10.0],
                           [10.0, 10.0, -10.0],
                           [10.0, -10.0, -10.0],
                           [10.0, -10.0, 10.0],
                           [-10.0, 10.0, 10.0],
                           [-10.0, 10.0, -10.0],
                           [-10.0, -10.0, -10.0],
                           [-10.0, -10.0, 10.0]])
# 绘制正方体12轴
line_pairs = [[0, 1], [1, 2], [2, 3], [3, 0],
              [4, 5], [5, 6], [6, 7], [7, 4],
              [0, 4], [1, 5], [2, 6], [3, 7]]

def get_head_pose(shape):# 头部姿态估计
    # （像素坐标集合）填写2D参考点，注释遵循https://ibug.doc.ic.ac.uk/resources/300-W/
    # 17左眉左上角/21左眉右角/22右眉左上角/26右眉右上角/36左眼左上角/39左眼右上角/42右眼左上角/
    # 45右眼右上角/31鼻子左上角/35鼻子右上角/48左上角/54嘴右上角/57嘴中央下角/8下巴角
    image_pts = np.float32([shape[17], shape[21], shape[22], shape[26], shape[36],
                            shape[39], shape[42], shape[45], shape[31], shape[35],
                            shape[48], shape[54], shape[57], shape[8]])
    # solvePnP计算姿势——求解旋转和平移矩阵：
    # rotation_vec表示旋转矩阵，translation_vec表示平移矩阵，cam_matrix与K矩阵对应，dist_coeffs与D矩阵对应。
    _, rotation_vec, translation_vec = cv2.solvePnP(object_pts, image_pts, cam_matrix, dist_coeffs)
    # projectPoints重新投影误差：原2d点和重投影2d点的距离（输入3d点、相机内参、相机畸变、r、t，输出重投影2d点）
    reprojectdst, _ = cv2.projectPoints(reprojectsrc, rotation_vec, translation_vec, cam_matrix,dist_coeffs)
    reprojectdst = tuple(map(tuple, reprojectdst.reshape(8, 2)))# 以8行2列显示

    # 计算欧拉角calc euler angle
    # 参考https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#decomposeprojectionmatrix
    rotation_mat, _ = cv2.Rodrigues(rotation_vec)#罗德里格斯公式（将旋转矩阵转换为旋转向量）
    pose_mat = cv2.hconcat((rotation_mat, translation_vec))# 水平拼接，vconcat垂直拼接
    # decomposeProjectionMatrix将投影矩阵分解为旋转矩阵和相机矩阵
    _, _, _, _, _, _, euler_angle = cv2.decomposeProjectionMatrix(pose_mat)
    
    pitch, yaw, roll = [math.radians(_) for _ in euler_angle]
 
 
    pitch = math.degrees(math.asin(math.sin(pitch)))
    roll = -math.degrees(math.asin(math.sin(roll)))
    yaw = math.degrees(math.asin(math.sin(yaw)))
    print('pitch:{}, yaw:{}, roll:{}'.format(pitch, yaw, roll))

    return reprojectdst, euler_angle# 投影误差，欧拉角

def eye_aspect_ratio(eye):
    # 垂直眼标志（X，Y）坐标
    A = dist.euclidean(eye[1], eye[5])# 计算两个集合之间的欧式距离
    B = dist.euclidean(eye[2], eye[4])
    # 计算水平之间的欧几里得距离
    # 水平眼标志（X，Y）坐标
    C = dist.euclidean(eye[0], eye[3])
    # 眼睛长宽比的计算
    ear = (A + B) / (2.0 * C)
    # 返回眼睛的长宽比
    return ear
 
def mouth_aspect_ratio(mouth):# 嘴部
    A = np.linalg.norm(mouth[2] - mouth[9])  # 51, 59
    B = np.linalg.norm(mouth[4] - mouth[7])  # 53, 57
    C = np.linalg.norm(mouth[0] - mouth[6])  # 49, 55
    mar = (A + B) / (2.0 * C)
    return mar

# 定义常数
# 眼睛长宽比
# 闪烁阈值
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3
# 打哈欠长宽比
# 闪烁阈值
MAR_THRESH = 0.5
MOUTH_AR_CONSEC_FRAMES = 3
# 瞌睡点头
HAR_THRESH = 0.3
NOD_AR_CONSEC_FRAMES = 3
# 初始化帧计数器和眨眼总数
COUNTER = 0
TOTAL = 0
# 初始化帧计数器和打哈欠总数
mCOUNTER = 0
mTOTAL = 0
# 初始化帧计数器和点头总数
hCOUNTER = 0
hTOTAL = 0

# 初始化DLIB的人脸检测器（HOG），然后创建面部标志物预测
print("[INFO] loading facial landmark predictor...")
# 第一步：使用dlib.get_frontal_face_detector() 获得脸部位置检测器
detector = dlib.get_frontal_face_detector()
# 第二步：使用dlib.shape_predictor获得脸部特征位置检测器
predictor = dlib.shape_predictor('D:/myworkspace/JupyterNotebook/fatigue_detecting/model/shape_predictor_68_face_landmarks.dat')
 
# 第三步：分别获取左右眼面部标志的索引
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# 第四步：打开cv2 本地摄像头
cap = cv2.VideoCapture(0)
 
# 从视频流循环帧
while True:
    # 第五步：进行循环，读取图片，并对图片做维度扩大，并进灰度化
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=720)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 第六步：使用detector(gray, 0) 进行脸部位置检测
    rects = detector(gray, 0)
    
    # 第七步：循环脸部位置信息，使用predictor(gray, rect)获得脸部特征位置的信息
    for rect in rects:
        shape = predictor(gray, rect)
        
        # 第八步：将脸部特征信息转换为数组array的格式
        shape = face_utils.shape_to_np(shape)
        
        # 第九步：提取左眼和右眼坐标
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        # 嘴巴坐标
        mouth = shape[mStart:mEnd]        
        
        # 第十步：构造函数计算左右眼的EAR值，使用平均值作为最终的EAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        # 打哈欠
        mar = mouth_aspect_ratio(mouth)
 
        # 第十一步：使用cv2.convexHull获得凸包位置，使用drawContours画出轮廓位置进行画图操作
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
 
        # 第十二步：进行画图操作，用矩形框标注人脸
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 1)    
 
        '''
            分别计算左眼和右眼的评分求平均作为最终的评分，如果小于阈值，则加1，如果连续3次都小于阈值，则表示进行了一次眨眼活动
        '''
        # 第十三步：循环，满足条件的，眨眼次数+1
        if ear < EYE_AR_THRESH:# 眼睛长宽比：0.2
            COUNTER += 1
           
        else:
            # 如果连续3次都小于阈值，则表示进行了一次眨眼活动
            if COUNTER >= EYE_AR_CONSEC_FRAMES:# 阈值：3
                TOTAL += 1
            # 重置眼帧计数器
            COUNTER = 0
            
        # 第十四步：进行画图操作，同时使用cv2.putText将眨眼次数进行显示
        cv2.putText(frame, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)     
        cv2.putText(frame, "COUNTER: {}".format(COUNTER), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (450, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        
        '''
            计算张嘴评分，如果小于阈值，则加1，如果连续3次都小于阈值，则表示打了一次哈欠，同一次哈欠大约在3帧
        '''
        # 同理，判断是否打哈欠    
        if mar > MAR_THRESH:# 张嘴阈值0.5
            mCOUNTER += 1
            cv2.putText(frame, "Yawning!", (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # 如果连续3次都小于阈值，则表示打了一次哈欠
            if mCOUNTER >= MOUTH_AR_CONSEC_FRAMES:# 阈值：3
                mTOTAL += 1
            # 重置嘴帧计数器
            mCOUNTER = 0
        cv2.putText(frame, "COUNTER: {}".format(mCOUNTER), (150, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
        cv2.putText(frame, "MAR: {:.2f}".format(mar), (300, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Yawning: {}".format(mTOTAL), (450, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        """
        瞌睡点头
        """
        # 第十五步：获取头部姿态
        reprojectdst, euler_angle = get_head_pose(shape)
        
        har = euler_angle[0, 0]# 取pitch旋转角度
        if har > HAR_THRESH:# 点头阈值0.3
            hCOUNTER += 1
        else:
            # 如果连续3次都小于阈值，则表示瞌睡点头一次
            if hCOUNTER >= NOD_AR_CONSEC_FRAMES:# 阈值：3
                hTOTAL += 1
            # 重置点头帧计数器
            hCOUNTER = 0
        
        # 绘制正方体12轴
        for start, end in line_pairs:
            cv2.line(frame, reprojectdst[start], reprojectdst[end], (0, 0, 255))
        # 显示角度结果
        cv2.putText(frame, "X: " + "{:7.2f}".format(euler_angle[0, 0]), (10, 90), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), thickness=2)# GREEN
        cv2.putText(frame, "Y: " + "{:7.2f}".format(euler_angle[1, 0]), (150, 90), cv2.FONT_HERSHEY_SIMPLEX,0.75, (255, 0, 0), thickness=2)# BLUE
        cv2.putText(frame, "Z: " + "{:7.2f}".format(euler_angle[2, 0]), (300, 90), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), thickness=2)# RED    
        cv2.putText(frame, "Nod: {}".format(hTOTAL), (450, 90),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        
            
        # 第十六步：进行画图操作，68个特征点标识
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

    print('嘴巴实时长宽比:{:.2f} '.format(mar)+"\t是否张嘴："+str([False,True][mar > MAR_THRESH]))
    print('眼睛实时长宽比:{:.2f} '.format(ear)+"\t是否眨眼："+str([False,True][COUNTER>=1]))
    
    # 确定疲劳提示:眨眼50次，打哈欠15次，瞌睡点头15次
    if TOTAL >= 50 or mTOTAL>=15 or hTOTAL>=15:
        cv2.putText(frame, "SLEEP!!!", (100, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
        
    # 按q退出
    cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
    # 窗口显示 show with opencv
    cv2.imshow("Frame", frame)
    
    # if the `q` key was pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# 释放摄像头 release camera
cap.release()
# do a bit of cleanup
cv2.destroyAllWindows()


[INFO] loading facial landmark predictor...
pitch:-4.804883946492844, yaw:5.770244360102852, roll:7.74171155417443
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.12 	是否眨眼：True
pitch:-5.787747731528659, yaw:7.175109500430537, roll:7.394245394316196
嘴巴实时长宽比:0.48 	是否张嘴：False
眼睛实时长宽比:0.10 	是否眨眼：True
pitch:-5.175370255050864, yaw:7.91738675335358, roll:7.44049987457066
嘴巴实时长宽比:0.46 	是否张嘴：False

D:\mydownload\Anaconda3\lib\site-packages\ipykernel_launcher.py:257: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index



眼睛实时长宽比:0.17 	是否眨眼：True
pitch:-3.417337430548953, yaw:7.882616332864543, roll:7.586535310282108
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:-3.6201771284730215, yaw:7.340429220026476, roll:7.7019484999778784
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.25 	是否眨眼：False
pitch:-2.9474014580855474, yaw:6.6989419328776725, roll:7.581566025809672
嘴巴实时长宽比:0.47 	是否张嘴：False
眼睛实时长宽比:0.28 	是否眨眼：False
pitch:-2.656883270625584, yaw:6.891667613620012, roll:7.770281951775834
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.28 	是否眨眼：False
pitch:-2.5584633180673033, yaw:7.228725418651386, roll:7.55791260229841
嘴巴实时长宽比:0.47 	是否张嘴：False
眼睛实时长宽比:0.29 	是否眨眼：False
pitch:-1.6390507723713066, yaw:7.588171158524226, roll:7.99493997075737
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.30 	是否眨眼：False
pitch:-2.7424520985192005, yaw:8.184204950605924, roll:7.58847447388287
嘴巴实时长宽比:0.47 	是否张嘴：False
眼睛实时长宽比:0.32 	是否眨眼：False
pitch:-1.8510234679978292, yaw:7.08853268938259, roll:7.681401909936735
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.32 	是否眨眼：F

嘴巴实时长宽比:0.49 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:2.1640461269786195, yaw:2.497989840677362, roll:1.0498757588609398
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.34 	是否眨眼：False
pitch:2.5014349049770304, yaw:2.671849540137353, roll:1.2209232568504962
嘴巴实时长宽比:0.48 	是否张嘴：False
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:1.828714747099065, yaw:2.131304328913043, roll:1.0999126841478388
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:1.828714747099065, yaw:2.131304328913043, roll:1.0999126841478388
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:0.8704423917093208, yaw:2.3971580293256003, roll:0.9695064946506641
嘴巴实时长宽比:0.48 	是否张嘴：False
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:-0.7183990634910293, yaw:2.4668343011557017, roll:1.2606030215773458
嘴巴实时长宽比:0.48 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:-4.452712493248596, yaw:2.445938636671752, roll:2.175793283208623
嘴巴实时长宽比:0.39 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:-4.452712493248596, yaw:2.445938636671752, roll:2.175793283208623
嘴巴实时长宽比:0.39 	是否张

嘴巴实时长宽比:1.13 	是否张嘴：True
眼睛实时长宽比:0.32 	是否眨眼：False
pitch:-5.929788847167021, yaw:-12.296890123340576, roll:0.8482172015975361
嘴巴实时长宽比:1.13 	是否张嘴：True
眼睛实时长宽比:0.32 	是否眨眼：False
pitch:-6.336235738026652, yaw:-9.326307359755196, roll:0.4787423979454993
嘴巴实时长宽比:1.09 	是否张嘴：True
眼睛实时长宽比:0.34 	是否眨眼：False
pitch:-5.432763616930323, yaw:-13.052237032356054, roll:0.8400767569303286
嘴巴实时长宽比:1.13 	是否张嘴：True
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:-5.427747710599182, yaw:-12.76107132281295, roll:0.9967912462542358
嘴巴实时长宽比:1.13 	是否张嘴：True
眼睛实时长宽比:0.37 	是否眨眼：False
pitch:-5.427747710599182, yaw:-12.76107132281295, roll:0.9967912462542358
嘴巴实时长宽比:1.13 	是否张嘴：True
眼睛实时长宽比:0.37 	是否眨眼：False
pitch:-5.50112345510213, yaw:-12.960871462911976, roll:1.051135959524049
嘴巴实时长宽比:1.14 	是否张嘴：True
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:-5.960422017711407, yaw:-10.603326753072432, roll:0.8883995638543116
嘴巴实时长宽比:1.09 	是否张嘴：True
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:-5.663602434496508, yaw:-9.07146464469925, roll:1.1636413113355255
嘴巴实时长宽比:1.07 	是

...

pitch:-2.1257432230949447, yaw:-4.45686348450669, roll:2.566966374056959
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.25 	是否眨眼：False
pitch:-2.2725207333961355, yaw:-3.4968355341831954, roll:2.7841436828330863
嘴巴实时长宽比:0.47 	是否张嘴：False
眼睛实时长宽比:0.25 	是否眨眼：False
pitch:-3.7604570938781006, yaw:-0.853227066007444, roll:2.907969890557501
嘴巴实时长宽比:0.46 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:-2.782477022126894, yaw:-0.5141777382613718, roll:2.8706617669951995
嘴巴实时长宽比:0.48 	是否张嘴：False
眼睛实时长宽比:0.14 	是否眨眼：True
pitch:-2.6599032477771862, yaw:1.882489323717419, roll:2.431587267988505
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:-2.6599032477771862, yaw:1.882489323717419, roll:2.431587267988505
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:-3.8212595017244024, yaw:1.8861729853335103, roll:3.242400507649001
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.25 	是否眨眼：False
pitch:-4.321882825679526, yaw:1.3558293249106894, roll:3.603310186442939
嘴巴实时长宽比:0.47 	是否张嘴：False
眼睛实时长宽比:0.29 	是否眨眼：False
pitch:-3.163

pitch:11.667599141651522, yaw:2.8397490414813893, roll:1.332343418627282
嘴巴实时长宽比:0.27 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:11.602184101501617, yaw:2.9302701982959967, roll:1.1437911167294628
嘴巴实时长宽比:0.27 	是否张嘴：False
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:12.288637131605661, yaw:2.8447187877105278, roll:1.9385426121290072
嘴巴实时长宽比:0.27 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:11.449924227866399, yaw:1.390935946265687, roll:2.689295989763751
嘴巴实时长宽比:0.26 	是否张嘴：False
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:12.585001392741752, yaw:2.8106691335867575, roll:2.549783584315975
嘴巴实时长宽比:0.27 	是否张嘴：False
眼睛实时长宽比:0.34 	是否眨眼：False
pitch:12.466307974390979, yaw:2.51354466242453, roll:2.467803018516919
嘴巴实时长宽比:0.27 	是否张嘴：False
眼睛实时长宽比:0.34 	是否眨眼：False
pitch:12.79758974690897, yaw:2.456954438791919, roll:2.4013699431187474
嘴巴实时长宽比:0.26 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:11.702170069321097, yaw:2.550063072302235, roll:1.9979715508269396
嘴巴实时长宽比:0.26 	是否张嘴：False
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:13.083349264

嘴巴实时长宽比:0.28 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
嘴巴实时长宽比:0.28 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
嘴巴实时长宽比:0.28 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
嘴巴实时长宽比:0.28 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
嘴巴实时长宽比:0.28 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:10.903902299274353, yaw:4.076042980355035, roll:3.9664300931655516
嘴巴实时长宽比:0.27 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:11.26656615220645, yaw:4.670576361383971, roll:3.8896744563890397
嘴巴实时长宽比:0.26 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:17.285014087202164, yaw:6.574287835461907, roll:3.969930432515831
嘴巴实时长宽比:0.26 	是否张嘴：False
眼睛实时长宽比:0.38 	是否眨眼：False
pitch:18.36908466361312, yaw:7.08561547784575, roll:3.9675652494604554
嘴巴实时长宽比:0.27 	是否张嘴：False
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:12.03675922202305, yaw:4.838736020595933, roll:2.8110548501439885
嘴巴实时长宽比:0.29 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:16.1799213665829, yaw:6.077641875500236, roll:2.8965312182195473
嘴巴实时长宽比:0.29 	是否张嘴：False
眼睛实时长宽比:0.36 	是否眨眼：False
pitch:10.1407506193

嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.37 	是否眨眼：False
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.37 	是否眨眼：False
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.37 	是否眨眼：False
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.37 	是否眨眼：False
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.37 	是否眨眼：False
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.37 	是否眨眼：False
pitch:1.8521464068275841, yaw:1.119071365105067, roll:4.497182188989733
嘴巴实时长宽比:0.41 	是否张嘴：False
眼睛实时长宽比:0.28 	是否眨眼：False
pitch:1.9861580254623448, yaw:-3.6645948621339763, roll:4.546271953491689
嘴巴实时长宽比:0.40 	是否张嘴：False
眼睛实时长宽比:0.28 	是否眨眼：False
嘴巴实时长宽比:0.40 	是否张嘴：False
眼睛实时长宽比:0.28 	是否眨眼：False
嘴巴实时长宽比:0.40 	是否张嘴：False
眼睛实时长宽比:0.28 	是否眨眼：False
嘴巴实时长宽比:0.40 	是否张嘴：False
眼睛实时长宽比:0.28 	是否眨眼：False
pitch:0.9242882113812868, yaw:1.511788157474865, roll:4.075046757876162
嘴巴实时长宽比:0.41 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
嘴巴实时长宽比:0.41 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
嘴巴实时长宽比:0.41 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
嘴巴实时长宽比:0.41 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
pitch:-0.6671640045696996, yaw:1

pitch:-6.06424233255625, yaw:-0.496184074710685, roll:11.947421628189225
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.34 	是否眨眼：False
pitch:-5.787737100590489, yaw:-0.8018382248998328, roll:11.844743297470549
嘴巴实时长宽比:0.44 	是否张嘴：False
眼睛实时长宽比:0.32 	是否眨眼：False
pitch:-6.35648095822131, yaw:-0.5642456099322397, roll:11.817948422820965
嘴巴实时长宽比:0.45 	是否张嘴：False
眼睛实时长宽比:0.32 	是否眨眼：False
pitch:-6.479920675679233, yaw:-0.4653772901933706, roll:11.58063116164624
嘴巴实时长宽比:0.42 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:-6.479920675679233, yaw:-0.4653772901933706, roll:11.58063116164624
嘴巴实时长宽比:0.42 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:-6.082688460152064, yaw:-0.41046005409779673, roll:11.974842997377248
嘴巴实时长宽比:0.44 	是否张嘴：False
眼睛实时长宽比:0.34 	是否眨眼：False
pitch:-7.1095416554661615, yaw:-2.211663088592992, roll:11.321032514501324
嘴巴实时长宽比:0.43 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pitch:-5.955356079932244, yaw:-0.36737157957613387, roll:11.679033645241109
嘴巴实时长宽比:0.42 	是否张嘴：False
眼睛实时长宽比:0.33 	是否眨眼：False
pit

In [18]:
import numpy as np
x=np.array([5,3,0])
y=np.array([4,4,0])
# 两个向量
Lx=np.sqrt(x.dot(x))
Ly=np.sqrt(y.dot(y))
#相当于勾股定理，求得斜线的长度
cos_angle=x.dot(y)/(Lx*Ly)#求得cos_sita的值再反过来计算，绝对长度乘以cos角度为矢量长度
angle=np.arccos(cos_angle)
angle2=angle*360/2/np.pi#变为角度
print(angle2)
#x.dot(y) =  y=∑(ai*bi)


14.036243467926484


In [16]:
import numpy as np
a = np.array([0,1,0])
b = np.array([0,0,0])
c = np.array([1,0,0])
# 两个向量
Lba = np.sqrt((a-b).dot(a-b))
Lbc = np.sqrt((c-b).dot(c-b))
cos_angle = (a-b).dot(c-b)/(Lba*Lbc)#相当于勾股定理，求得斜线的长度
angle = np.arccos(cos_angle)#求得cos_sita的值再反过来计算，绝对长度乘以cos角度为矢量长度
angle2=angle*360/2/np.pi#变为角度
print(angle2)

90.0
